<a href="https://colab.research.google.com/github/rishuraj27/Practice/blob/master/lower_price_predictor_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

In [89]:
pip install xgboost

In [48]:
train_df = pd.read_csv("/content/Train.csv")
test_df = pd.read_csv("/content/Test.csv")

In [49]:
train_df["Date"]=pd.to_datetime(train_df["Date"])
test_df["Date"]=pd.to_datetime(test_df["Date"])

In [50]:
#train_df.loc[:,"year"]=train_df["Date"].dt.year
train_df.loc[:,"month"]=train_df["Date"].dt.month
train_df.loc[:,"day"]=train_df["Date"].dt.day
#train_df.loc[:,"weekday"]=train_df["Date"].dt.dayofweek
#train_df.loc[:,"quarter"]=train_df["Date"].dt.quarter
#test_df.loc[:,"year"]=test_df["Date"].dt.year
test_df.loc[:,"month"]=test_df["Date"].dt.month
test_df.loc[:,"day"]=test_df["Date"].dt.day
#test_df.loc[:,"weekday"]=test_df["Date"].dt.dayofweek
#test_df.loc[:,"quarter"]=test_df["Date"].dt.quarter

In [5]:
print(train_df.shape)
print(test_df.shape)

(9798, 11)
(5763, 10)


In [31]:
train_df.corr()

,State_of_Country,Market_Category,Product_Category,Grade,Demand,Low_Cap_Price,High_Cap_Price
State_of_Country,1.000000,0.122124,0.205858,-0.038140,-0.002647,0.234901,0.027959
Market_Category,0.122124,1.000000,-0.030353,-0.034454,-0.049241,0.198524,0.144557
Product_Category,0.205858,-0.030353,1.000000,-0.056394,0.085874,0.180240,0.114415
Grade,-0.038140,-0.034454,-0.056394,1.000000,0.047414,-0.079639,0.171083
Demand,-0.002647,-0.049241,0.085874,0.047414,1.000000,-0.057482,-0.033971
Low_Cap_Price,0.234901,0.198524,0.180240,-0.079639,-0.057482,1.000000,0.721021
High_Cap_Price,0.027959,0.144557,0.114415,0.171083,-0.033971,0.721021,1.000000


In [52]:
print(train_df["Product_Category"].value_counts())
print("**")
print(test_df["Product_Category"].value_counts())

0     7167
10    1050
1      524
13     408
12     204
15     168
16      79
11      62
14      43
7       30
5       23
6       18
3        9
4        5
17       4
9        2
8        2
Name: Product_Category, dtype: int64
**
0     2826
10    1228
1      608
13     512
11     138
12     122
14      95
15      66
6       44
16      40
5       33
7       23
3       11
2        9
9        5
17       2
4        1
Name: Product_Category, dtype: int64


In [51]:
for i in train_df.columns:
  print(i,len(train_df[i].unique()))
print("**")
for i in test_df.columns:
  print(i,len(test_df[i].unique()))

Item_Id 9798
Date 1070
State_of_Country 21
Market_Category 275
Product_Category 17
Grade 4
Demand 1207
Low_Cap_Price 5392
High_Cap_Price 6468
month 12
day 31
**
Item_Id 5763
Date 617
State_of_Country 20
Market_Category 240
Product_Category 17
Grade 4
Demand 860
High_Cap_Price 4781
month 12
day 31


In [54]:
print(train_df.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9798 entries, 0 to 9797
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Item_Id           9798 non-null   object        
 1   Date              9798 non-null   datetime64[ns]
 2   State_of_Country  9798 non-null   int64         
 3   Market_Category   9798 non-null   int64         
 4   Product_Category  9798 non-null   int64         
 5   Grade             9798 non-null   int64         
 6   Demand            9798 non-null   float64       
 7   Low_Cap_Price     9798 non-null   int64         
 8   High_Cap_Price    9798 non-null   int64         
 9   month             9798 non-null   int64         
 10  day               9798 non-null   int64         
 11  weekday           9798 non-null   int64         
 12  quarter           9798 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(10), object(1)
memory usage: 995.2+ KB

In [8]:
train_df.head()

,Item_Id,Date,State_of_Country,Market_Category,Product_Category,Grade,Demand,Low_Cap_Price,High_Cap_Price,month,day
0,IT_1,2007-07-05,0,0,0,0,0.5,2785,4240,7,5
1,IT_2,2007-07-05,0,1,0,0,0.7,3574,4756,7,5
2,IT_3,2007-07-05,0,103,0,1,1.6,5978,9669,7,5
3,IT_4,2007-07-05,0,103,0,0,0.0,5681,8313,7,5
4,IT_5,2007-07-05,0,103,0,2,0.0,4924,7257,7,5


In [53]:
numerical= ["Demand","High_Cap_Price"] 
dep = ['Low_Cap_Price']
date_=["Date"]
integer =['High_Cap_Price']
categorical = ["State_of_Country","Market_Category","Product_Category","Grade"]
cc=["Grade",'month','day']

In [10]:
# scaler = StandardScaler()
# train_df[numerical] = scaler.fit_transform(train_df[numerical])
# test_df[numerical]=scaler.fit_transform(test_df[numerical])

## Encoding categorical variable

In [54]:
#categorical.remove("Employee_ID")
train_df= pd.get_dummies(train_df,columns= cc)
test_df= pd.get_dummies(test_df,columns= cc)

In [55]:
print(train_df.shape)
print(test_df.shape)

(9798, 55)
(5763, 54)


## dependent and independent variable

In [56]:
item_id_id=train_df[["Item_Id"]]
X=train_df.drop(["Item_Id","Low_Cap_Price","Date"],axis=1)
y=train_df[["Low_Cap_Price"]]
df_test = test_df.drop(["Item_Id","Date"],axis=1) 
#X = scaler.fit_transform(X)
print(X.shape)
print(df_test.shape)

(9798, 52)
(5763, 52)


In [67]:
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor,ExtraTreesRegressor,RandomForestClassifier,RandomForestRegressor
#from sklearn.linear_model import BayesianRidge
#dt = DecisionTreeRegressor(random_state=3)
#dt = DecisionTreeClassifier(random_state=3)
#dt = RandomForestClassifier(random_state=3)
dt = RandomForestRegressor(random_state=3)
#dt = XGBRegressor(random_state=3)
#dt= ExtraTreesRegressor(max_depth=2,min_samples_leaf=.22,random_state=3)
#dt=BayesianRidge(compute_score=True)
#abd_reg = AdaBoostRegressor(base_estimator=dt,n_estimators=100)
#sgbt=GradientBoostingRegressor(max_depth=1,subsample=0.8,max_features=0.1,n_estimators=250,random_state=3)

In [19]:
# param_dt = {"max_depth":[1,2,3,8,9,10,15,20],
#             "min_samples_leaf":[0.2,0.22,.24,.3,.33],
#             "max_features":[0.6,0.8,1]
#             }
# grid_dt = GridSearchCV(estimator=dt,param_grid=param_dt,scoring='neg_mean_squared_log_error',cv=10)

In [ ]:
# # Hyperparameter tunning for BR
# param_dt = {"n_iter":[10,50,100,200,300,400,500],
#             # "tol":[10**(-x) for x in range(3,7)],
#             # "alpha_1":[10**(-x) for x in range(4,9)],
#             # "alpha_2": [10**(-x) for x in range(4,9)],
#             # "lambda_1" : [10**(-x) for x in range(4,9)],
#             # "lambda_2": [10**(-x) for x in range(4,9)]
#             }
# grid_dt = GridSearchCV(estimator=dt,param_grid=param_dt,scoring='neg_root_mean_squared_error',cv=5)

In [60]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=3)

In [13]:
# import sklearn
# sklearn.metrics.SCORERS.keys()

In [68]:
import warnings
warnings.filterwarnings("ignore")
#y_train = np.round(y_train)
# dt.fit(X_train,y_train)
#dt.fit(X_train,y_train)
dt.fit(X,y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=3, verbose=0, warm_start=False)

In [110]:
# grid_dt.fit(X,y)

In [17]:
# best_param=grid_dt.best_params_
# best_param

In [16]:
# b_e = grid_dt.best_estimator_
# b_e

In [18]:
# grid_dt.best_score_

In [17]:
from sklearn.metrics import mean_squared_log_error 

In [70]:
score=max(0,(100-mean_squared_log_error(y_test,y_pred)))
print(score)

99.96826045565338


In [69]:
y_pred= dt.predict(X_test)
min(y_pred)
max(y_pred)

14954.74

In [71]:
print(np.max(y_test))
print(np.min(y_test))
print(np.max(y_pred))
print(np.min(y_pred))

Low_Cap_Price    17580
dtype: int64
Low_Cap_Price    1
dtype: int64
14954.74
140.85


In [67]:
# mse_dt=MSE(y_test,y_pred)
# rmse_dt = np.sqrt(mse_dt)
# rmse_dt

In [45]:
train_df.columns

Index(['Item_Id', 'Date', 'State_of_Country', 'Market_Category', 'Demand',
       'Low_Cap_Price', 'High_Cap_Price', 'Product_Category_0',
       'Product_Category_1', 'Product_Category_3', 'Product_Category_4',
       'Product_Category_5', 'Product_Category_6', 'Product_Category_7',
       'Product_Category_8', 'Product_Category_9', 'Product_Category_10',
       'Product_Category_11', 'Product_Category_12', 'Product_Category_13',
       'Product_Category_14', 'Product_Category_15', 'Product_Category_16',
       'Product_Category_17', 'Grade_0', 'Grade_1', 'Grade_2', 'Grade_3',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7', 'day_8',
       'day_9', 'day_10', 'day_11', 'day_12', 'day_13', 'day_14', 'day_15',
       'day_16', 'day_17', 'day_18', 'day_19', 'day_20', 'day_21', 'day_22',
       'day_23', 'day_24', 'day_25

In [46]:
test_df.columns

Index(['Item_Id', 'Date', 'State_of_Country', 'Market_Category', 'Demand',
       'High_Cap_Price', 'Product_Category_0', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3', 'Product_Category_4',
       'Product_Category_5', 'Product_Category_6', 'Product_Category_7',
       'Product_Category_9', 'Product_Category_10', 'Product_Category_11',
       'Product_Category_12', 'Product_Category_13', 'Product_Category_14',
       'Product_Category_15', 'Product_Category_16', 'Product_Category_17',
       'Grade_0', 'Grade_1', 'Grade_2', 'Grade_3', 'month_1', 'month_2',
       'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8',
       'month_9', 'month_10', 'month_11', 'month_12', 'day_1', 'day_2',
       'day_3', 'day_4', 'day_5', 'day_6', 'day_7', 'day_8', 'day_9', 'day_10',
       'day_11', 'day_12', 'day_13', 'day_14', 'day_15', 'day_16', 'day_17',
       'day_18', 'day_19', 'day_20', 'day_21', 'day_22', 'day_23', 'day_24',
       'day_25', 'day_26', 'day

In [72]:
#test_df_final = test_df.drop(["Item_Id",'Date'],axis =1)
y_test_pred = dt.predict(df_test)

In [73]:
dd = {"Item_Id":test_df["Item_Id"], "Low_Cap_Price":y_test_pred}
result = pd.DataFrame(dd)

result.set_index('Item_Id', inplace=True)

In [74]:
result.to_csv("/content/result_06.csv")

In [ ]:
score = 100* max(0, 1-rmse_dt )
score

80.7505357914807

In [29]:
# # Neural Net
# import tensorflow as tf
# # GRADED FUNCTION: train_mnist
# def train_mnist():
#     # Please write your code only where you are indicated.
#     # please do not remove # model fitting inline comments.

#     # YOUR CODE SHOULD START HERE
#     class myCallback(tf.keras.callbacks.Callback):
#         def on_epoch_end(self,epoch,log={}):
#             if log.get("acc")>0.99:
#                 print("/n Reached 99% accuracy so cancelling training/n")
#                 self.model.stop_training = True
#     # YOUR CODE SHOULD END HERE
    
#     callbacks = myCallback()
#     mnist = tf.keras.datasets.mnist

#     (x_train, y_train),(x_test, y_test) = mnist.load_data(path=path)
#     # YOUR CODE SHOULD START HERE
#     x_train = x_train/255.0
#     x_test = x_test/255.0
#     # YOUR CODE SHOULD END HERE
#     model = tf.keras.models.Sequential([
#         # YOUR CODE SHOULD START HERE
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512,activation = tf.nn.relu),
#     tf.keras.layers.Dense(10, activation = tf.nn.softmax)
#         # YOUR CODE SHOULD END HERE
#     ])

#     model.compile(optimizer='adam',
#                   loss='sparse_categorical_crossentropy',
#                   metrics=['accuracy'])
    
#     # model fitting
#     history = model.fit(# YOUR CODE SHOULD START HERE
#         x_train,y_train,epochs=10,callbacks =[callbacks]
#               # YOUR CODE SHOULD END HERE
#     )
#     # model fitting
#     return history.epoch, history.history['acc'][-1]


# # GRADED FUNCTION: house_model
# def house_model(y_new):
#     xs = np.array([float(x) for x in range(1,10)])
#     ys = np.array([0.5+0.5*i for i in range(1,10)])
#     model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
#     model.compile(optimizer = "sgd", loss = "mean_squared_error")
#     model.fit(xs,ys,epochs=500)
#     return model.predict(y_new)[0]

In [65]:
# # GRADED FUNCTION: train_mnist_conv
# def train_mnist_conv():
#     # Please write your code only where you are indicated.
#     # please do not remove model fitting inline comments.

#     # YOUR CODE STARTS HERE
#     class myCallback(tf.keras.callbacks.Callback):
#         def on_epoch_end(self,epoch,log={}):
#             if log.get("acc") >0.998:
#                 print("\n Reached 99.8% acuracy so cancelling trainig!")
#                 self.model.stop_training= True
                
#     callbacks = myCallback()
#     # YOUR CODE ENDS HERE

#     mnist = tf.keras.datasets.mnist
#     (training_images, training_labels), (test_images, test_labels) = mnist.load_data(path=path)
#     # YOUR CODE STARTS HERE
#     training_images= training_images.reshape(60000,28,28,1)
#     test_images = test_images.reshape(10000,28,28,1)
#     training_images = training_images/255.0
#     test_images = test_images/255.0
#     # YOUR CODE ENDS HERE

#     model = tf.keras.models.Sequential([
#             # YOUR CODE STARTS HERE
#         tf.keras.layers.Conv2D(64,(3,3),activation ="relu",input_shape=(28,28,1)),
#         tf.keras.layers.MaxPooling2D(2,2),
#         tf.keras.layers.Flatten(),
#         tf.keras.layers.Dense(128,activation="relu"),
#         tf.keras.layers.Dense(10,activation="softmax")
#             # YOUR CODE ENDS HERE
#     ])

#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#     # model fitting
#     history = model.fit(
#         # YOUR CODE STARTS HERE
#         training_images,training_labels,epochs=10,callbacks =[callbacks]
#         # YOUR CODE ENDS HERE
#     )
#     # model fitting
#     return history.epoch, history.history['acc'][-1]



In [66]:
import tensorflow as tf